In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'svg'

## Import module

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
import numpy as np
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image

import sys
sys.path.append("../../../src/")
sys.path.append("../../")
import model
from datasets import imagenet
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map
from utils.visualizations.visualize import concat_imgs, preprocess_arrays

## Hyperparameter

In [3]:
num_class = 30
class_index = 950


selected_layer = "relu"
selected_filter = 5
exp = "052400"
epoch = "80175"

exp = "052417"
epoch = "151660"
selected_filter = 17
selected_layer = "layer1.0.relu"

exp = "052415"
epoch = "200430"
selected_filter = 17
selected_layer = "layer2.0.relu"

exp = "052416"
epoch = "191701"
selected_filter = 37
selected_layer = "layer1.1.relu"

In [4]:
backbone = "resnet18"
color_map = "nipy_spectral"
model_dir = "/home/lincolnzjx/Desktop/Interpretation/saved/models"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Load modal from dict

In [5]:
# model
net = model.Network(backbone=backbone, num_classes=1000, pretrained=True) 
net.to(device)
net.eval()

=> ResNet18 using pretrained: True


Network(
  (model): ResNet18(
    (resnet18): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm

In [6]:
summary(net, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

## Load Original Data 

In [7]:
# Load data
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
reverse_mean = [-0.485, -0.456, -0.406]
reverse_std = [1/0.229, 1/0.224, 1/0.225]
train_transform = None
train_transform = transforms.Compose([                                                                                                                                
    transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
    transforms.ToTensor(),                                                                                                                                            
    #transforms.Normalize(mean, std)                                                                                                                                   
       ])         
trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", 
                             is_train=True, transform=train_transform)

trainset.set_data([class_index], num_class)
imgs_path = []                                                                                                                                                            
images = []
labels = []
for img, label, img_path in trainset:                                                                                                                                     
    images.append(img.unsqueeze(0))                                                                                                                                       
    labels.append(label)                                                                                                                                                  
    imgs_path.append(img_path)  

Len of new dataset is :30


## Create some need path

In [8]:
save_dir = "../../../saved/pack/"
################### Hyper-Parameter #######################
# exp = resume_exp
# epoch = resume_epoch
##########################################################
ab_path = os.path.join(save_dir, exp, epoch)

In [9]:
index2image = {index: item.split("/")[-1].split(".")[0] for index, item in enumerate(imgs_path)}
index2image

{0: 'n07747607_10061',
 1: 'n07747607_10068',
 2: 'n07747607_10110',
 3: 'n07747607_10134',
 4: 'n07747607_10162',
 5: 'n07747607_10166',
 6: 'n07747607_10178',
 7: 'n07747607_10179',
 8: 'n07747607_10184',
 9: 'n07747607_10192',
 10: 'n07747607_10204',
 11: 'n07747607_10205',
 12: 'n07747607_1021',
 13: 'n07747607_10213',
 14: 'n07747607_10217',
 15: 'n07747607_10232',
 16: 'n07747607_10258',
 17: 'n07747607_10290',
 18: 'n07747607_10305',
 19: 'n07747607_10345',
 20: 'n07747607_10358',
 21: 'n07747607_10379',
 22: 'n07747607_10420',
 23: 'n07747607_10425',
 24: 'n07747607_10436',
 25: 'n07747607_10465',
 26: 'n07747607_10482',
 27: 'n07747607_10485',
 28: 'n07747607_10491',
 29: 'n07747607_10545'}

In [10]:
images_cpu = np.array([image.detach().clone().cpu().numpy().squeeze() for image in images])

In [11]:
# Load image
optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True)
valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
optimized_data_zscore = zscore(optimized_data, mean, std)
images_zscore = zscore(images_cpu, mean, std)

Load from ../../../saved/pack/052416/191701/n07747607_10061.png
Load from ../../../saved/pack/052416/191701/n07747607_10068.png
Load from ../../../saved/pack/052416/191701/n07747607_10110.png
Load from ../../../saved/pack/052416/191701/n07747607_10134.png
Load from ../../../saved/pack/052416/191701/n07747607_10162.png
Load from ../../../saved/pack/052416/191701/n07747607_10166.png
Load from ../../../saved/pack/052416/191701/n07747607_10178.png
Load from ../../../saved/pack/052416/191701/n07747607_10179.png
Load from ../../../saved/pack/052416/191701/n07747607_10184.png
Load from ../../../saved/pack/052416/191701/n07747607_10192.png
Load from ../../../saved/pack/052416/191701/n07747607_10204.png
Load from ../../../saved/pack/052416/191701/n07747607_10205.png
Load from ../../../saved/pack/052416/191701/n07747607_1021.png
Load from ../../../saved/pack/052416/191701/n07747607_10213.png
Load from ../../../saved/pack/052416/191701/n07747607_10217.png
Load from ../../../saved/pack/052416/1917

In [12]:
# Move to device
opt_image = torch.from_numpy(optimized_data_zscore).to(device)
original_image = torch.from_numpy(images_zscore).to(device)

## Obtain feature from conv layers.

In [13]:
ori_activation_maps = net.get_activation_maps(original_image, 
                                              selected_layer=selected_layer)[0]
opt_activation_maps = net.get_activation_maps(opt_image, 
                                              selected_layer=selected_layer)[0]

=> Register fhook layer1.1.relu
=> Register fhook layer1.1.relu


In [14]:
images_cpu_p, optimized_data_cpu, ori_activation_maps_cpu, opt_activation_maps_cpu =\
    preprocess_arrays(images_cpu, optimized_data,
                      ori_activation_maps[:, selected_filter], 
                     opt_activation_maps[:, selected_filter])
concated_imgs = concat_imgs(images_cpu_p, optimized_data_cpu,
                            ori_activation_maps_cpu, opt_activation_maps_cpu)

## Visualization Here.

In [15]:
save_path = os.path.join("../../../saved/pack/", exp, 
                         "concat.png")
concated_imgs.save(save_path)